# 1.Loading JSON dataset

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.scripts.glove2word2vec import glove2word2vec
from sklearn.metrics.pairwise import cosine_similarity
from gensim.models import KeyedVectors
from nltk.corpus import stopwords
from textblob import TextBlob
from textblob import Word
import pandas as pd 
import numpy as np 
import nltk
import json
import os
import re

def read_truth(path):
    data = []
    index = []
    for i, line in enumerate(open(path + '/truth.jsonl', 'r')):
        instance = json.loads(line)
        data.append(instance)
        index.append(instance['id'])
    df = pd.DataFrame(data=data, index=index)
    df.sort_index(inplace=True)
    return df

def read_instance(path):
    data = []
    index = []
    for i, line in enumerate(open(path + '/instances.jsonl', 'rb')):
        instance = json.loads(line)
        data.append(instance)
        index.append(instance['id'])
    df = pd.DataFrame(data=data, index=index)
    df.sort_index(inplace=True)
    return df


train_truth = read_truth('./clickbait17-train-170331')
train_instances = read_instance('./clickbait17-train-170331')
validation_truth = read_truth('./clickbait17-validation-170630')
validation_instances = read_instance('./clickbait17-validation-170630')

D:\Anaconda\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
train = pd.merge(train_truth, train_instances, on = 'id')
validation = pd.merge(validation_truth, validation_instances, on = 'id')
data = pd.concat([train, validation],ignore_index = True)
data['truthClass'] = data['truthClass'].map({'clickbait':True ,'no-clickbait':False}).astype(bool)

# 2.Basic Feature Extraction

## 2.1 number of words

In [3]:
def avg_word(sentence):
    sentence = str(sentence)
    words = sentence.split()
    return round((sum(len(word) for word in words)/len(words)), 1)


char_title = []
word_title = []
avg_paragraph = []
char_paragraph = []
word_paragraph = []
char_caption = []
word_caption = []

for title in data['targetTitle']:
    char = len(str(title))
    word = len(str(title).split(' '))
    char_title.append(char)
    word_title.append(word)
    
for paragraph in data['targetParagraphs']:
    char = len(str(paragraph))
    word = len(str(paragraph).split(' '))
    avg = avg_word(paragraph)
    avg_paragraph.append(avg)
    char_paragraph.append(char)
    word_paragraph.append(word)
    
for caption in data['targetCaptions']:
    char = len(str(caption))
    word = len(str(caption).split(' '))
    char_caption.append(char)
    word_caption.append(word)

In [4]:
char_feature = pd.DataFrame([char_caption,
                             word_caption,
                             char_title,
                             word_title,
                             char_paragraph,
                             word_paragraph,
                             avg_paragraph])
char_feature = char_feature.T
char_feature.rename(columns = {0: 'char_caption',
                               1: 'word_caption',
                               2: 'char_title',
                               3: 'word_title',
                               4: 'char_paragraph',
                               5: 'word_paragraph',
                               6: 'avg_paragraph',
                              },inplace = True)

In [5]:
char_feature.describe()

,char_caption,word_caption,char_title,word_title,char_paragraph,word_paragraph,avg_paragraph
count,21997.000000,21997.000000,21997.000000,21997.000000,21997.000000,21997.000000,21997.000000
mean,585.354730,83.002182,80.443333,13.201755,3637.643406,594.734464,5.224603
std,2605.827071,329.696620,152.086847,23.953162,4226.469811,706.772198,1.348661
min,2.000000,1.000000,4.000000,1.000000,2.000000,1.000000,2.000000
25%,39.000000,4.000000,56.000000,9.000000,1464.000000,237.000000,5.000000
50%,142.000000,20.000000,67.000000,11.000000,2650.000000,429.000000,5.200000
75%,458.000000,69.000000,80.000000,13.000000,4643.000000,757.000000,5.400000
max,144852.000000,16316.000000,4038.000000,651.000000,199672.000000,33306.000000,112.000000


## 2.2 Text feature

In [6]:
num_at = []
num_acronym = []
num_baity = []#!
num_cap = []
num_digit = []
num_exclm = []
num_money = []
num_ques = []
num_quote = []#!
num_tag = []
num_pic = []
num_parenthesis = []
is_start_num = []
is_superlative = []
is_start5w1h = []

table_currency = ['¥','$','€','£','￠']
table_bracket = ['(',')','[',']','{','}']
table_quote = ["'m","'re","'ve","'d","'s","s'"]
table_baity = ["click here","exclusive","won't believe","happen next","don't want","you know"]
table_5w1h = ["what","why","when","who","which","how"]

for caption in data['targetCaptions']:
    num_pic.append(len(set(caption)))  

for sentences in data['targetTitle']:
    sentences = str(sentences)
    tmp = sentences.split()
    s = str(tmp[0])
    
    question_mark = 0
    start_digit = False
    start_5w1h = False
    parenthesis = 0
    superlative = False
    exclamation = 0
    digital = 0
    acronym = 0
    capital = 0
    money = 0
    baity = 0
    quote = 0
    start = 0
    hash_tag = 0
    at = 0
    
    text = nltk.word_tokenize(sentences)
    part_of_speech = nltk.pos_tag(text)
    for token,tag in part_of_speech:
        if tag in ['RBS','JJS']:
            superlative  = True
            
    if s in table_5w1h:
        start_5w1h = True
    if s.isdigit():
        start_digit = True
        
    for word in sentences.split():        
        if len(word) <= 5:
            acronym += 1
        if len(word) > 5:
            if word.isupper():
                capital += 1                
        for char in word:
            if char == '!':
                exclamation += 1
            if char == '?':
                question_mark += 1
            if char in table_currency:
                money += 1
            if char in table_bracket:
                parenthesis += 1
            if char == '@':
                at += 1
            if char == '#':
                hash_tag += 1
            if char.isdigit:
                digital += 1
    
    num_acronym.append(acronym)
    num_at.append(at)
    num_cap.append(capital)
    num_digit.append(digital)
    num_exclm.append(exclamation)
    num_money.append(money)
    num_parenthesis.append(parenthesis)
    num_ques.append(question_mark)
    num_tag.append(hash_tag)
    is_start5w1h.append(start_5w1h)
    is_start_num.append(start_digit)
    is_superlative.append(superlative)
#     num_baity.append(baity)
#     num_quote.append(quote)


In [7]:
title_feature = pd.DataFrame([num_acronym,
                             num_at,
                             num_cap,
                             num_digit,
                             num_exclm,
                             num_money,
                             num_parenthesis,
                             num_ques,
                             num_tag,
                             num_pic,
                             is_start_num, 
                             is_superlative,
#                              is_start5w1h
                               ])
title_feature = title_feature.T
title_feature.rename(columns = {0:'num_acronym',
                               1:'num_at',
                               2:'num_cap',
                               3:'num_digit',
                               4:'num_exclm',
                               5:'num_money',
                               6:'num_parenthesis',
                               7:'num_ques',
                               8:'num_tag',
                               9:'num_pic',
                               10:'is_start_num',  
                               11:'is_superlative',
#                                10:'is_start5w1h'
                              },inplace = True)

In [8]:
title_feature.describe()

,num_acronym,num_at,num_cap,num_digit,num_exclm,num_money,num_parenthesis,num_ques,num_tag,num_pic,is_start_num,is_superlative
count,21997,21997,21997,21997,21997,21997,21997,21997,21997,21997,21997,21997
unique,81,2,4,264,4,5,3,7,3,98,2,2
top,6,0,0,53,0,0,0,0,0,1,False,False
freq,2996,21988,21599,656,21513,21398,21688,20897,21941,6495,21095,21009


## 2.3 Informality and Forward Reference features
+ CLScore = 0.0588*L - 0.296*S - 15.8  

   L = average number of letters     
   S = average number of sentence per 100 words  
   

+ RIX = LW/S  

+ LIX = W/S + (100*LW)/W  

   W = number of words     
   LW number of long words(7+ characters)     
   S = number of sentence  
   
   
+ Formality Measure (fmeasure):  

   (nounfreq+adjectivefreq+prepositionfreq+particlefreq-pronounfreq-verbfreq-adverbfreq-interjectionfreq+100)*0.5   
   

+ Sentiment Analysis:  

   extract polarity as it indicates the sentiment as value nearer to 1 means a positive sentiment and values nearer to -1 means a negative sentiment.

In [9]:
RIX = []
LIX = []
sentiment = []
CLScore = []
f_measure = []

for paragraph in data['targetParagraphs']:
    
    avg_letter = 0
    avg_sentence = 0
    num_word = 0
    num_char = 0
    num_sentence = 0
    long_word = 0
    noun_freq = 0
    adjective_freq = 0
    preposition_freq = 0
    particle_freq = 0
    pronoun_freq = 0
    verb_freq = 0
    adverb_freq = 0
    interjection_freq = 0  
    measure = 0
    
    paragraph = str(paragraph)
    text = nltk.word_tokenize(paragraph)
    part_of_speech = nltk.pos_tag(text)
    for token,tag in part_of_speech:
        if tag in ['NN','NNS','NNP','NNPS']:
            noun_freq += 1
        if tag in ['VB','VBD','VBG','VBN','VBP','VBZ']:
            verb_freq += 1
        if tag == 'UH':
            interjection_freq += 1
        if tag in ['RB','RBS','RBR']:
            adverb_freq += 1
        if tag == 'RP':
            particle_freq += 1
        if tag in ['JJ','JJR','JJS'] :
            adjective_freq += 1
        if tag == 'in':
            preposition_freq += 1
        if tag in ['WRB','WP$','WP','PRP$','PRP']:
            pronoun_freq += 1
    measure = (noun_freq+adjective_freq+preposition_freq+particle_freq-pronoun_freq-verb_freq-adverb_freq-interjection_freq)
        
    for word in paragraph.split():
        num_word += 1
        num_char += len(word)
        if len(word) >= 7:
            long_word += 1
        for char in word:
            if char == '.':
                num_sentence += 1
    
    h_word = num_word//100
    if h_word == 0:
        h_word = 1
    if num_sentence == 0:
        num_sentence = 1
    if num_word == 0:
        num_word = 1

    avg_letter = round((num_char / num_word), 1)
    avg_sentence = round((num_sentence / h_word), 1)
    RIX.append(round(long_word/num_sentence, 1))
    CLScore.append(round(0.0588*avg_letter-0.296*avg_sentence-15.8, 1))
    sentiment.append(round(TextBlob(paragraph).sentiment[0], 1))
    LIX.append(round(num_word/num_sentence + (100*long_word)/num_word, 1))
    f_measure.append((measure+100)/2)
    

In [10]:
content_feature = pd.DataFrame([RIX,
                             LIX,
                             sentiment,
                             CLScore,
                             f_measure,
                               ])
content_feature = content_feature.T
content_feature.rename(columns = {0:'RIX',
                               1:'LIX',
                               2:'sentiment',
                               3:'CLScore',
                               4:'f_measure',
                              },inplace = True)

In [11]:
content_feature.describe()

,RIX,LIX,sentiment,CLScore,f_measure
count,21997.000000,21997.000000,21997.000000,21997.000000,21997.000000
mean,5.565254,47.946152,0.095872,-17.396790,96.145906
std,3.774795,12.432667,0.110498,0.930461,69.958521
min,0.000000,1.000000,-1.000000,-35.200000,-521.500000
25%,4.000000,42.300000,0.000000,-17.700000,63.000000
50%,5.200000,47.500000,0.100000,-17.300000,81.500000
75%,6.500000,52.500000,0.100000,-16.900000,111.000000
max,142.000000,515.200000,1.000000,-10.100000,3373.500000


## 2.4 Similarity between Title and Top 5 Sentences

In [12]:
top1sim = []
top2sim = []
top3sim = []
top4sim = []
top5sim = []

for i in range(len(data)):
    
    title = data['targetTitle'][i]
    body = data['targetParagraphs'][i]
    sim_1 = sim_2 = sim_3 = sim_4 = sim_5 = 0.0
    sen_1 = sen_2 = sen_3 = sen_4 = sen_5 = ''
    
    if len(body) == 0:
        top1sim.append(sim_1)
        top2sim.append(sim_2)
        top3sim.append(sim_3)
        top4sim.append(sim_4)
        top5sim.append(sim_5)
        continue
    if len(body) >= 1:
        sen_1 = body[0]
    if len(body) >= 2:
        sen_2 = body[1]
    if len(body) >= 3:
        sen_3 = body[2]
    if len(body) >= 4:
        sen_4 = body[3]
    if len(body) >= 5:
        sen_5 = body[4]
    
    essay = str(title + ' ' + sen_1 + ' ' + sen_2 + ' '  + sen_3 + ' ' + sen_4 + ' ' + sen_5)
    tfidf = TfidfVectorizer(lowercase = True, analyzer = 'word', stop_words = 'english',ngram_range = (1,1))
    train_vect = tfidf.fit_transform([essay])    
    title_vect = tfidf.transform([title])
    
    s1_vect = tfidf.transform([sen_1])
    sim_1 = round(float(cosine_similarity(title_vect,s1_vect)),1)
    top1sim.append(sim_1)
    
    s2_vect = tfidf.transform([sen_2])
    sim_2 = round(float(cosine_similarity(title_vect,s2_vect)),1)
    top2sim.append(sim_2)
    
    s3_vect = tfidf.transform([sen_3])
    sim_3 = round(float(cosine_similarity(title_vect,s3_vect)),1)
    top3sim.append(sim_3)
    
    s4_vect = tfidf.transform([sen_4])
    sim_4 = round(float(cosine_similarity(title_vect,s4_vect)),1)
    top4sim.append(sim_4)
    
    s5_vect = tfidf.transform([sen_5])
    sim_5 = round(float(cosine_similarity(title_vect,s5_vect)),1)
    top5sim.append(sim_5)

In [13]:
sim_feature = pd.DataFrame([top1sim,
                             top2sim,
                             top3sim,
                             top4sim,
                             top5sim,
                               ])
sim_feature = sim_feature.T
sim_feature.rename(columns = {0:'top1sim',
                               1:'top2sim',
                               2:'top3sim',
                               3:'top4sim',
                               4:'top5sim',
                              },inplace = True)

In [14]:
sim_feature.describe()

,top1sim,top2sim,top3sim,top4sim,top5sim
count,21997.000000,21997.000000,21997.000000,21997.00000,21997.000000
mean,0.237628,0.166332,0.126017,0.11778,0.100714
std,0.208294,0.166482,0.139019,0.13668,0.122919
min,0.000000,0.000000,0.000000,0.00000,0.000000
25%,0.000000,0.000000,0.000000,0.00000,0.000000
50%,0.200000,0.100000,0.100000,0.10000,0.100000
75%,0.400000,0.300000,0.200000,0.20000,0.200000
max,1.000000,1.000000,1.000000,1.00000,1.000000


# 3.Basic Pre-processing

## 3.1 Lower case Removing Punctuation Removal of Stop Words

In [15]:
en_corpus = stopwords.words('english')
stop_words = []
caption = []
content = []
title = []

for sentence in data['targetParagraphs']:
    sentence = str(sentence).replace('[^\w\s]','')
    sentence = re.sub('[\u0060|\u0021-\u002c|\u002e-\u002f|\u003a-\u003f|\u2200-\u22ff|\uFB00-\uFFFD|\u2E80-\u33FF]',' ',sentence)
    stop_cnt = 0
    string = ''
#     sentence = TextBlob(sentence).correct()
    for word in sentence.split():
        if word in en_corpus:
            stop_cnt += 1
        if word not in en_corpus:            
            string += ' ' + word.lower()
    stop_words.append(stop_cnt)
    content.append(string)    

for sentence in data['targetCaptions']:
    string = ''
    sentence = str(sentence).replace('[^\w\s]','')
    sentence = re.sub('[\u0060|\u0021-\u002c|\u002e-\u002f|\u003a-\u003f|\u2200-\u22ff|\uFB00-\uFFFD|\u2E80-\u33FF]',' ',sentence)
    for word in sentence.split():
        if word not in en_corpus:            
            string += ' ' + word.lower()
    caption.append(string) 

for sentence in data['targetTitle']:
    string = ''
    sentence = str(sentence).replace('[^\w\s]','')
    sentence = re.sub('[\u0060|\u0021-\u002c|\u002e-\u002f|\u003a-\u003f|\u2200-\u22ff|\uFB00-\uFFFD|\u2E80-\u33FF]',' ',sentence)
    for word in sentence.split():
        if word not in en_corpus:            
            string += ' ' + word.lower()
    title.append(string)
    
corpus = title + content + caption

## 3.2 Common / Rare words removal & Lemmatization

In [16]:
target_caption = []
target_content = []
target_title = []

common = pd.Series(' '.join(caption).split()).value_counts()[:10]
rare = pd.Series(' '.join(caption).split()).value_counts()[-10:]  

for sentence in caption:
    string = ''
    lemma = ''
    sentence = str(sentence)
    for word in sentence.split():
        if word not in common:
            if word  not in rare:
                lemma = Word(word).lemmatize()
                string += ' ' + lemma
    target_caption.append(string)

common = pd.Series(' '.join(content).split()).value_counts()[:10]
rare = pd.Series(' '.join(content).split()).value_counts()[-10:]  

for sentence in content:
    string = ''
    sentence = str(sentence)
    for word in sentence.split():
        if word not in common:
            if word  not in rare:
                lemma = Word(word).lemmatize()
                string += ' ' + lemma
    target_content.append(string)
    
common = pd.Series(' '.join(title).split()).value_counts()[:10]
rare = pd.Series(' '.join(title).split()).value_counts()[-10:]  

for sentence in title:
    string = ''
    sentence = str(sentence)
    for word in sentence.split():
        if word not in common:
            if word  not in rare:
                lemma = Word(word).lemmatize()
                string += ' ' + lemma
    target_title.append(string) \
    
target_corpus = []
for i in range(len(target_content)):
    total_context = str(target_caption[i] + target_content[i] + target_title[i])
    target_corpus.append(total_context)

# 4.Advance Text Processing
## 4.1 N-grams

In [17]:
TextBlob(corpus[0]).ngrams(2)

[WordList(['tony', 'nominees']),
 WordList(['nominees', 'craziest']),
 WordList(['craziest', 'moments']),
 WordList(['moments', 'stage'])]

## 4.2 TF-IDF

In [18]:
tfidf = TfidfVectorizer(max_features = 1024, lowercase = True, analyzer = 'word', stop_words = 'english',ngram_range = (1,1))
corpus_vect = tfidf.fit_transform(target_corpus)

vec_feature = []

for article in target_corpus:
    corpus_vect = tfidf.transform([article])
    vec_feature.append(corpus_vect)

## 4.3 BoW

In [19]:
bow = CountVectorizer(max_features=1000, lowercase=True, ngram_range=(1,1),analyzer = "word")
train_bow = bow.fit_transform(target_corpus)

train_bow

<21997x1000 sparse matrix of type '<class 'numpy.int64'>'
	with 2535921 stored elements in Compressed Sparse Row format>

## 4.4 Word Embeddings

In [20]:
glove_input_file = 'glove.840B.300d.txt'
word2vec_output_file = 'glove.840B.300d.txt.word2vec'
glove2word2vec(glove_input_file, word2vec_output_file)
filename = 'glove.840B.300d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)

In [21]:
filename = 'glove.840B.300d.txt.word2vec'
model = KeyedVectors.load_word2vec_format(filename, binary=False)

# 5.Model

In [27]:
import matplotlib.pyplot as plt 
import seaborn as sns

from sklearn.model_selection import StratifiedShuffleSplit
from sklearn import metrics
from sklearn.metrics import accuracy_score, log_loss
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.linear_model import LogisticRegression

classifiers = [
    KNeighborsClassifier(3),
    SVC(probability=True),
    DecisionTreeClassifier(),
    RandomForestClassifier(),
    AdaBoostClassifier(),
    GradientBoostingClassifier(),
    GaussianNB(),
    LinearDiscriminantAnalysis(),
    QuadraticDiscriminantAnalysis(),
    LogisticRegression()]
classifiers_name = [
    "KNeighborsClassifier",
    "SVC",
   " DecisionTreeClassifier",
    "RandomForestClassifier",
    "AdaBoostClassifier",
    "GradientBoostingClassifier",
    "GaussianNB",
    "LinearDiscriminantAnalysis",
    "QuadraticDiscriminantAnalysis",
    "LogisticRegression"]

log_cols = ['Classifier','Accuracy']
log = pd.DataFrame(columns = log_cols)

sss = StratifiedShuffleSplit(n_splits = 10, test_size = 0.1, random_state = 0)

acc_dict = []

handy_feature = pd.concat([char_feature,title_feature,content_feature,sim_feature], axis = 1)
X = handy_feature
y = data['truthMean']

for train_index, test_index in sss.split(X,y):
    X_train, X_test = X[train_index],y[test_index]
    y_train, y_test = y[train_index],y[test_index]
    
    for clf in classsifiers:
        name = clf.__class__.name__
        clf.fit(X_train,y_train)
        acc = accuracy_score(y_test,train_predictions)
        if name in acc_dict:
            acc_dict[name] += acc
        else:
            acc_dict[name] = acc
        
for clf in acc_dict:
    acc_dict[clf] = acc_dict
    log_entry = pd.DataFrame([[clf,acc_dict[clf]]],columns = log_cols)
    
print(log)

KeyError: '[ 8027 17745 15991 ...  1281 14078 20125] not in index'